# Sandy waterfalls

* https://adventofcode.com/2022/day/14

Today's puzzle references back to [2018 Day 17](../2018/Day%2017.ipynb), with good reason! This is, essentially, the exact same problem but with added diagonals as the sand piles up. Except, that makes the problem entirely different.

In this case, it is actually easier to just keep a set of obstacle coordinates. Sand follows a path past the obstacles until it comes to rest, adding to the obstacles. At some point, sand passes below the lowest `y` coordinate, at which point you are done.

In [1]:
from collections.abc import Collection
from dataclasses import dataclass
from itertools import count, product
from typing import Iterable, NamedTuple, Self, TypeAlias, assert_never


class Pos(NamedTuple):
    x: int
    y: int

    def __add__(self, other: Self) -> Self:
        return Pos(self.x + other.x, self.y + other.y)


@dataclass
class Cave:
    rock: set[Pos]

    @classmethod
    def from_rock_lines(cls, lines: Iterable[str]) -> Self:
        rock = set()
        for line in lines:
            coords = iter(map(int, coord.split(",")) for coord in line.split("->"))
            fx, fy = next(coords)
            for tx, ty in coords:
                dx, dy = 1 if tx > fx else -1, 1 if ty > fy else -1
                rock.update(
                    Pos(x, y)
                    for x, y in product(range(fx, tx + dx, dx), range(fy, ty + dy, dy))
                )
                fx, fy = tx, ty
        return cls(rock)

    def simulate_sand(self, with_floor: bool = False) -> int:
        occupied = set(self.rock)
        # set the boundaries; min_y is the lowest non-occupied point below the origin
        # and max_y is the point at which grains will enter free-fall, or for part 2,
        # hit the floor.
        min_y, max_y = min(p.y for p in occupied), max(p.y for p in occupied)
        if with_floor:
            max_y += 1  # floor is at +2, but we test for y > max_y
        directions: tuple[Pos, Pos, Pos] = (Pos(0, 1), Pos(-1, 1), Pos(1, 1))
        for grains in count():
            sand = Pos(500, min_y)
            if sand in occupied:
                return grains
            while True:
                for d in directions:
                    if (new := sand + d) not in occupied:
                        if new.y <= max_y:
                            sand = new
                            break
                        if not with_floor:
                            return grains
                else:
                    min_y = min(min_y, max(0, sand.y - 1))
                    occupied.add(sand)
                    break
        assert_never("Can't ever get here")


example = Cave.from_rock_lines(
    ["498,4 -> 498,6 -> 496,6", "503,4 -> 502,4 -> 502,9 -> 494,9"]
)
assert example.simulate_sand() == 24

In [2]:
import aocd


cave = Cave.from_rock_lines(aocd.get_data(day=14, year=2022).splitlines())
print("Part 1:", cave.simulate_sand())

Part 1: 683


## Part 2 bottoms out

In part 2, we need to add an infinite floor past our maximum Y coordinate. I've refactored my code to update the maximum by 1 as that means the new sand grain coordinate would have to penetrate this infinitely-wide floor at that point. I also needed to add a new check for the grain not having moved, and we are done! Just take into account that we need to return the current grain counter plus 1, as it is the next grain that'll be blocked.

To speed things up a little, I added a mininum Y coordinate that grains need to fall from; this cuts out a considerable amount of looping almost every grain has to go through.

In [3]:
assert example.simulate_sand(with_floor=True) == 93

In [4]:
print("Part 2:", cave.simulate_sand(with_floor=True))

Part 2: 28821
